In [1]:
from keras.models import load_model
from collections import deque
import numpy as np
import pickle
import cv2
import os

Using TensorFlow backend.


In [2]:
model = load_model(r"KD_classification_model\classification_model")
lb = pickle.loads(open(r"KD_classification_model\KD_classification_binarizer.pickle","rb").read())
#video_output = r"KD_classification_model\classified_output.mp4"

mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
queue = deque(maxlen=50)


In [3]:
capture_video = cv2.VideoCapture(r"KD_classification_model\WhatsApp_Video.mp4")
writer = None
(width, height) = (None, None)


save_path = os.path.join("output_frames", "")
xx = 1
yy = 0
zz = 0


while True:
    (taken, frame) = capture_video.read()
    
    if not taken:
        print("All Frames Done")
        break
    if width is None or height is None:
        (width, height) = frame.shape[:2]
    
    
    # Keep a copy of the frame
    output = frame.copy()
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224,224)).astype("float32")
    frame -= mean
    
    # Predict each frame
    pred = model.predict(np.expand_dims(frame, axis=0))[0]
    #print(pred)
    
    # Rolling Averaging the prediction
    queue.append(pred)
    result = np.array(queue).mean(axis=0)
    i = np.argmax(result)
    label = lb.classes_[i]
    
    #print(i)
    
    '''
    # Count kill and death score
    if (xx % 50 == 0):
        if (i==0):
            yy = yy+1
        else:
            zz = zz+1
    '''
    
    # Count kill and death score
    
    if(xx % 500 == 0 and i==1):
        zz = zz+1
    
    if(xx % 500 == 0 and i==0):
        yy = yy+1
    
    
    # Write the predicted lable on the original copied frame
    text1 = "Score : {}".format(label)
    cv2.putText(output, text1, (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
    text2 = "Kill Count : {}".format(zz)
    cv2.putText(output, text2, (25,50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
    text3 = "Death Count : {}".format(yy)
    cv2.putText(output, text3, (25,75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)
    
    
    # Save the frames
    filename = save_path + str(xx) + ".png"
    cv2.imwrite(filename, output)
    xx = xx+1
    
    
    # Show running frames
    cv2.imshow("In progress frame by frame", output)
    
    key = cv2.waitKey(1) & 0xFF
        
    if key == ord("q"):
        break
    


print("Finalizing...")

capture_video.release()
cv2.destroyAllWindows()

print("Done")


All Frames Done
Finalizing...
Done


In [4]:
# Create the output video

import glob
import re
 
img_array = []
numbers = re.compile(r'(\d+)')

def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts


for filename in sorted(glob.glob('D:/Opus Project/output_frames/*.png'), key=numericalSort):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)


out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 50, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])

out.release()
